In [1]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

  Obtaining dependency information for PyPDF2 from https://files.pythonhosted.org/packages/8e/5e/c86a5643653825d3c913719e788e41386bee415c2b87b4f955432f2de6b2/pypdf2-3.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   ------ -------------------------------- 41.0/232.6 kB 487.6 kB/s eta 0:00:01
   ---------------------------- ----------- 163.8/232.6 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 1.4 MB/s eta 0:00:00
  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/3e/8f/55f8cd863c678a703aa3b97e5f4511cfd2c52987ba05459afdcdec76eeb3/faiss_cpu-1.7.4-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB 1.4 MB/s et

In [3]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [4]:
import os
openai_key=''
os.environ["OPENAI_API_KEY"] = openai_key


In [7]:
pdfreader=PdfReader('C:\\Users\\athre\\Downloads\\expdf.pdf')

In [8]:
from typing_extensions import Concatenate

raw_text=''
for i,page in enumerate(pdfreader.pages):
    content=page.extract_text()
    if content:
        raw_text+=content

In [9]:
raw_text

'U.S. Department of the Treasury Report\nSeptember 2022\nThe Future of Money  \nand Payments \nReport Pursuant to Section 4(b) of Executive Order 14067This page has been intentionally left blank.Table of Contents\nI. Introduction  ....................................................................................................... 1\nII. Money and payments  ......................................................................................... 3\nMoney, payments, and the current payment system  .................................................. 3\nConsumers and payments today  ............................................................................... 11\nRecent innovations in money and payments  ............................................................ 14\nIII. Potential U.S. CBDC design choices  ................................................................... 19\nMoney  .....................................................................................................

In [11]:
text_splitter=CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

texts=text_splitter.split_text(raw_text)

In [12]:
len(texts)

249

In [15]:
embeddings=OpenAIEmbeddings()

In [16]:
document_search=FAISS.from_texts(texts,embeddings)

In [17]:
document_search

In [18]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [25]:
chain=load_qa_chain(OpenAI(),chain_type="stuff")
query="When is a payment system secure"

docs=document_search.similarity_search(query)

result=chain.run(input_documents=docs,question=query)

In [26]:
result


' The payment system is secure when it has appropriate cybersecurity protection, a robust incident response program, and considers security and resilience in its design.'

In [27]:
chain2=load_qa_chain(OpenAI(),chain_type="stuff")
query="Why should the US promote inoovations in payments"

docs=document_search.similarity_search(query)

result=chain2.run(input_documents=docs,question=query)

In [28]:
result

' The US should promote innovations in payments because it can lead to a faster, cheaper, and more transparent international payments system, which can support economic growth and provide more rapid support for underserved communities. Additionally, innovations in payments can make the payment system more competitive, efficient, and inclusive.'